# 1.   PREPARACIÓN NOTEBOOK



---

## 1.1 Librerías

In [226]:
from google.colab import drive
import sys
import os
import shutil
import sqlite3 as sql #Crear y trabajar bases de datos ligeras
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from sklearn.preprocessing import StandardScaler
import joblib

## 1.2 Conexiones

### 1.2.1 Conexión al repositorio en Drive

In [227]:
#Conectar al drive local
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [228]:
#Conectar al repositorio
path = '/content/drive/My Drive/cod/A3_marketing'

sys.path.append(path) #Importar las funciones propias a través de import, porque incluye la carpeta del repositorio como uno de esos paquetes para que import busque funciones
os.chdir(path) #Subir y descargar archivos de la ruta del repositorio de trabajo

### 1.2.2 Notebook de funciones

In [229]:
%run a_funciones.ipynb

### 1.2.3 Copia de la bd para trabajar

In [230]:
# Definir rutas
DATABASE_ORIGINAL_PATH = '/content/drive/My Drive/cod/A3_marketing/data/db_movies'
DATABASE_COPY_PATH = '/content/drive/My Drive/cod/A3_marketing/data/db_movies_c'

#Crear una copia de la base de datos original para trabajar con las mismas marcas de tiempo y permisos
try:
    shutil.copy2(DATABASE_ORIGINAL_PATH, DATABASE_COPY_PATH)
    print(f"Se ha creado una copia de la base de datos en: {DATABASE_COPY_PATH}")
except FileNotFoundError:
    print(f"Error: No se encontró la base de datos original en: {DATABASE_ORIGINAL_PATH}")
    exit()
except Exception as e:
    print(f"Error al copiar la base de datos: {e}")
    exit()

Se ha creado una copia de la base de datos en: /content/drive/My Drive/cod/A3_marketing/data/db_movies_c


### 1.2.4 Conexión a la base de datos

In [231]:
#Conexión a la COPIA de la base de datos (función propia)
con, cur = conectar_bd(DATABASE_COPY_PATH)

Conexión exitosa a la base de datos (copia): /content/drive/My Drive/cod/A3_marketing/data/db_movies_c


# 2.   EXPLORACIÓN Y TRATAMIENTO

---

## 2.1 Exploración inicial

In [232]:
# Revisar cuántas tablas hay en la base de datos, cuáles son las columnas de estas tablas y cuántas filas tienen (función propia)

if cur:
    print('-' * 100)
    tablas = listar_tablas(cur)
    print('-' * 100)
    for tabla in tablas:
        if tabla in tablas:
            explorar_esquema_tabla(cur, tabla)
            contar_filas_tabla(cur, tabla)
            mostrar_primeras_filas_sql(cur, tabla)
        print('-' * 100)

----------------------------------------------------------------------------------------------------

Tablas encontradas en la base de datos:
* ratings
* movies
----------------------------------------------------------------------------------------------------

 ESQUEMA DE LA TABLA 'ratings'

  - Nombre Columna: userId
    Tipo de Dato: INTEGER
    ¿Permite Valores Nulos? (0 es sí - 1 es no): 0

  - Nombre Columna: movieId
    Tipo de Dato: INTEGER
    ¿Permite Valores Nulos? (0 es sí - 1 es no): 0

  - Nombre Columna: rating
    Tipo de Dato: REAL
    ¿Permite Valores Nulos? (0 es sí - 1 es no): 0

  - Nombre Columna: timestamp
    Tipo de Dato: INTEGER
    ¿Permite Valores Nulos? (0 es sí - 1 es no): 0

 NÚMERO TOTAL DE FILAS EN 'ratings': 100836

 PRIMERAS 5 FILAS DE LA TABLA 'ratings'
(1, 1, 4.0, 964982703)
(1, 3, 4.0, 964981247)
(1, 6, 4.0, 964982224)
(1, 47, 5.0, 964983815)
(1, 50, 5.0, 964982931)
----------------------------------------------------------------------------------

In [233]:
#Se consulta la cantidad de usuarios y de películas del sistema (función propia)

conteo_usuarios_peliculas(cur)

Número total de usuarios únicos: 610
Número total de películas únicas: 9724


## 2.2 Exploración tabla 'movies'

### 2.2.1 Extracción del año y limpieza de títulos

In [234]:
#Prueba o visualización de una de las filas antes de separar el año, correguir títulos y dividir los géneros (revisar por ejemplo el 32 por doble parentesis, el 27008)
cur.execute("SELECT * FROM movies WHERE movieId = ?", (27008,))
fila_testeo = cur.fetchone()
print(fila_testeo)

(27008, 'From Dusk Till Dawn 2: Texas Blood Money (1999) ', 'Comedy|Crime|Horror')


In [235]:
#Se separa en una columna nueva llamada 'year' lo que se asume como el año de lanzamiento de película, del título y
#se deja el título solo en la columna 'title'


# Se crea una nueva columna llamada 'year'
cur.execute("""
    ALTER TABLE movies ADD COLUMN year INTEGER;
""")
con.commit()
print("\nSe añadió la columna 'year' a la tabla 'movies'.")

# Extraer el año del título usando el último paréntesis en el título como referencia
#y teniendo en cuenta que algunos títulos tienen un esapcio después de este paréntesis
cur.execute("""
    UPDATE movies
    SET year = CAST(SUBSTR(TRIM(title), LENGTH(TRIM(title)) - 4, 4) AS INTEGER)
    WHERE TRIM(title) GLOB '*([1-2][0-9][0-9][0-9])*';
""")
con.commit()
print("\nSe extrajo el año de los títulos y se actualizó la columna 'year'.")

# Limpiar la columna 'title' del año y dejar solo el nombre de la película
cur.execute("""
    UPDATE movies
    SET title = TRIM(SUBSTR(title, 1, INSTR(title, '(' || year || ')') - 1))
    WHERE year IS NOT NULL AND INSTR(title, '(' || year || ')') > 0;
""")
con.commit()
print("\nSe eliminaron los años de los títulos'.")


Se añadió la columna 'year' a la tabla 'movies'.

Se extrajo el año de los títulos y se actualizó la columna 'year'.

Se eliminaron los años de los títulos'.


In [236]:
#Prueba de la misma fila anterior
cur.execute("SELECT * FROM movies WHERE movieId = ?", (27008,))
fila_testeo = cur.fetchone()
print(fila_testeo)

(27008, 'From Dusk Till Dawn 2: Texas Blood Money', 'Comedy|Crime|Horror', 1999)


In [237]:
# Mostrar las primeras filas para ver el resultado

cur.execute("SELECT * FROM movies LIMIT 10;")
primeras_filas_con_year = cur.fetchall()
if primeras_filas_con_year:
    print("\nPrimeras filas de la tabla 'movies':")
    for fila in primeras_filas_con_year:
        print(fila)
else:
    print("\nLa tabla está vacía")


Primeras filas de la tabla 'movies':
(1, 'Toy Story', 'Adventure|Animation|Children|Comedy|Fantasy', 1995)
(2, 'Jumanji', 'Adventure|Children|Fantasy', 1995)
(3, 'Grumpier Old Men', 'Comedy|Romance', 1995)
(4, 'Waiting to Exhale', 'Comedy|Drama|Romance', 1995)
(5, 'Father of the Bride Part II', 'Comedy', 1995)
(6, 'Heat', 'Action|Crime|Thriller', 1995)
(7, 'Sabrina', 'Comedy|Romance', 1995)
(8, 'Tom and Huck', 'Adventure|Children', 1995)
(9, 'Sudden Death', 'Action', 1995)
(10, 'GoldenEye', 'Action|Adventure|Thriller', 1995)


### 2.2.2 Validación de valores nulos

In [238]:
# Verificar valores nulos en cada columna de la tabla 'movies'
cur.execute("PRAGMA table_info('movies');")
columnas = cur.fetchall()

# Para cada columna, contar los valores nulos
for columna in columnas:
    nombre_columna = columna[1]
    cur.execute(f"SELECT COUNT(*) FROM movies WHERE {nombre_columna} IS NULL;")
    nulos = cur.fetchone()[0]

    if nulos > 0:
        print(f"La columna '{nombre_columna}' tiene {nulos} valor(es) nulo(s).")
    else:
        print(f"La columna '{nombre_columna}' no tiene valores nulos.")

La columna 'movieId' no tiene valores nulos.
La columna 'title' no tiene valores nulos.
La columna 'genres' no tiene valores nulos.
La columna 'year' tiene 13 valor(es) nulo(s).


### 2.2.3 Validación de datos 'no genres listed'

In [239]:
#Revisar los datos '(no genres listed)'
cur.execute("SELECT COUNT(*) FROM movies WHERE genres = '(no genres listed)';")
sin_genero = cur.fetchone()[0]
if sin_genero > 0:
    print(f"Hay {sin_genero} película(s) con '(no genres listed)' en la columna 'genres'.")

Hay 34 película(s) con '(no genres listed)' en la columna 'genres'.


In [240]:
# Imprimir los registros donde 'genres' es '(no genres listed)'
cur.execute("SELECT * FROM movies WHERE genres = '(no genres listed)';")
registros_no_genero = cur.fetchall()

if registros_no_genero:
    print("\nRegistros con '(no genres listed)' en la columna 'genres':")
    for registro in registros_no_genero:
        print(registro)


Registros con '(no genres listed)' en la columna 'genres':
(114335, 'La cravate', '(no genres listed)', 1957)
(122888, 'Ben-hur', '(no genres listed)', 2016)
(122896, 'Pirates of the Caribbean: Dead Men Tell No Tales', '(no genres listed)', 2017)
(129250, 'Superfast!', '(no genres listed)', 2015)
(132084, 'Let It Be Me', '(no genres listed)', 1995)
(134861, 'Trevor Noah: African American', '(no genres listed)', 2013)
(141131, 'Guardians', '(no genres listed)', 2016)
(141866, 'Green Room', '(no genres listed)', 2015)
(142456, 'The Brand New Testament', '(no genres listed)', 2015)
(143410, 'Hyena Road', '(no genres listed)', None)
(147250, 'The Adventures of Sherlock Holmes and Doctor Watson', '(no genres listed)', None)
(149330, 'A Cosmic Christmas', '(no genres listed)', 1977)
(152037, 'Grease Live', '(no genres listed)', 2016)
(155589, 'Noin 7 veljestä', '(no genres listed)', 1968)
(156605, 'Paterson', '(no genres listed)', None)
(159161, 'Ali Wong: Baby Cobra', '(no genres listed)',

### 2.2.4 Se eliminan las películas sin género

In [241]:
# Ejecutar la consulta para obtener las películas sin género listado
cur.execute("""
    SELECT movieId, title, genres FROM movies WHERE genres = '(no genres listed)';
""")
peliculas_sin_genero = cur.fetchall()
print(f"\nNúmero de películas sin género: {len(peliculas_sin_genero)}")
print('*' * 100)

if len(peliculas_sin_genero) > 0:
    print("\nPrimeras películas sin género listado: \n")
    for pelicula in peliculas_sin_genero[:5]:  # Mostrar solo las primeras 5
        print(pelicula)
    print('*' * 100)

# Eliminar las películas sin género listado de la base de datos
cur.execute("""
    DELETE FROM movies WHERE genres = '(no genres listed)';
""")
con.commit()
print("\nPelículas sin género eliminadas de la tabla 'movies'.")


Número de películas sin género: 34
****************************************************************************************************

Primeras películas sin género listado: 

(114335, 'La cravate', '(no genres listed)')
(122888, 'Ben-hur', '(no genres listed)')
(122896, 'Pirates of the Caribbean: Dead Men Tell No Tales', '(no genres listed)')
(129250, 'Superfast!', '(no genres listed)')
(132084, 'Let It Be Me', '(no genres listed)')
****************************************************************************************************

Películas sin género eliminadas de la tabla 'movies'.


### 2.2.5 Nulos en columna 'year' después de eliminar las películas sin género

In [242]:
# Revisar los registros con valores nulos en la columna 'year'
cur.execute("SELECT * FROM movies WHERE year IS NULL;")
registros_nulos_year = cur.fetchall()

# Imprimir los registros donde 'year' es nulo
if registros_nulos_year:
    print("\nRegistros con valores nulos en la columna 'year':")
    for registro in registros_nulos_year:
        print(registro)


Registros con valores nulos en la columna 'year':
(40697, 'Babylon 5', 'Sci-Fi', None)
(140956, 'Ready Player One', 'Action|Sci-Fi|Thriller', None)
(149334, 'Nocturnal Animals', 'Drama|Thriller', None)
(162414, 'Moonlight', 'Drama', None)


### 2.2.6 Separación de los géneros

In [243]:
#Se crea dataframe de pandas para hacer la división con esa librería
df_movies_processed = pd.read_sql('SELECT * FROM movies', con)

print("\nPrimeras filas de df_movies_processed (antes de separar géneros):\n")
df_movies_processed.head()


Primeras filas de df_movies_processed (antes de separar géneros):



,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men,Comedy|Romance,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0


In [244]:
# Separar los géneros con (función propia)
df_movies_processed = separar_generos(df_movies_processed.copy())

print("\nPrimeras filas de df_movies_processed (con géneros separados):\n")
df_movies_processed.head()


Primeras filas de df_movies_processed (con géneros separados):



,movieId,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995.0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995.0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995.0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995.0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,1995.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2.3 Exploración tabla 'ratings'

### 2.3.1 Validación valores nulos

In [245]:
df_ratings_processed = pd.read_sql('SELECT * FROM ratings', con)

In [246]:
# Verificar valores nulos en cada columna de la tabla 'ratings'
cur.execute("PRAGMA table_info('ratings');")
columnas = cur.fetchall()

# Para cada columna, contar los valores nulos
for columna in columnas:
    nombre_columna = columna[1]
    cur.execute(f"SELECT COUNT(*) FROM ratings WHERE {nombre_columna} IS NULL;")
    nulos = cur.fetchone()[0]

    if nulos > 0:
        print(f"La columna '{nombre_columna}' tiene {nulos} valor(es) nulo(s).")
    else:
        print(f"La columna '{nombre_columna}' no tiene valores nulos.")

La columna 'userId' no tiene valores nulos.
La columna 'movieId' no tiene valores nulos.
La columna 'rating' no tiene valores nulos.
La columna 'timestamp' no tiene valores nulos.


### 2.4 Se elimina la columna timestamp en la tabla 'ratings'

In [247]:
# Renombrar la tabla original
cur.execute("ALTER TABLE ratings RENAME TO ratings_old;")

# Crear una nueva tabla 'ratings' sin la columna 'timestamp'
cur.execute("""
    CREATE TABLE ratings (
        userId INTEGER NOT NULL,
        movieId INTEGER NOT NULL,
        rating REAL NOT NULL
    );
""")

# Copiar los datos a la nueva tabla
cur.execute("""
    INSERT INTO ratings (userId, movieId, rating)
    SELECT userId, movieId, rating FROM ratings_old;
""")

# Eliminar la tabla antigua
cur.execute("DROP TABLE ratings_old;")

# Confirmar cambios
con.commit()
print("Columna 'timestamp' eliminada correctamente de la tabla 'ratings'.")

Columna 'timestamp' eliminada correctamente de la tabla 'ratings'.


In [248]:
explorar_esquema_tabla(cur, 'ratings')


 ESQUEMA DE LA TABLA 'ratings'

  - Nombre Columna: userId
    Tipo de Dato: INTEGER
    ¿Permite Valores Nulos? (0 es sí - 1 es no): 1

  - Nombre Columna: movieId
    Tipo de Dato: INTEGER
    ¿Permite Valores Nulos? (0 es sí - 1 es no): 1

  - Nombre Columna: rating
    Tipo de Dato: REAL
    ¿Permite Valores Nulos? (0 es sí - 1 es no): 1


# 3. ANÁLISIS EXPLORATORIO (EDA)




---

## 3.1 Distribución de clafisifaciones

In [249]:
df_rating_dist = distribucion_calificaciones(cur)
print(df_rating_dist)

   rating  frecuencia
0     0.5        1370
1     1.0        2811
2     1.5        1791
3     2.0        7551
4     2.5        5550
5     3.0       20047
6     3.5       13136
7     4.0       26818
8     4.5        8551
9     5.0       13211


In [250]:
fig = px.bar(df_rating_dist, x='rating', y='frecuencia',
                             title='Distribución de Calificaciones',
                             labels={'rating': 'Calificación', 'frecuencia': 'Frecuencia'})

fig.update_layout(
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False))

fig.show()

El histograma muestra que la mayoria de las calificaciones se concentran entre 3.0 y 4.0 con un pico en la calificaicón 4.0, lo cual indica una tendencia positiva en la percepcion de las peliculas por parte de los usuarios. Las calificaciones bajas, entre 0 a 2.5, son menos frecuentes, lo que nos podria decir que los usuarios tienden a valorar mas positivamente las peliculas.

##3.2 Distribución de calificaciones por usuario

In [251]:
cur.execute("""
    SELECT userId, COUNT(*) as cnt_rating
    FROM ratings
    GROUP BY userId
""")
user_ratings = pd.DataFrame(cur.fetchall(), columns=['userId', 'cnt_rating'])

fig2 = px.histogram(user_ratings,
                   x='cnt_rating',
                   nbins=30,
                   title='Numero de calificaciones por usuario',
                   labels={'cnt_rating': 'Cantidad de calificaciones', 'count': 'Frecuencia de usuarios'})

fig2.update_layout(
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False))

fig2.show()

Mediante el gráfico de barras podemos observar que el pico de calificaciones por usuario en la plataforma está entre 0 y 99. De 100 calificaciones por usuario en adelante vemos una drastica reducción hasta llegar el maximo de un solo usuario con un total de entre 2600 a 2699 peliculas calificadas.

## 3.3 Distribución de calificaciones por película

In [252]:
cur.execute("""
    SELECT movieId, COUNT(*) as cnt_rating
    FROM ratings
    GROUP BY movieId
""")
movie_ratings = pd.DataFrame(cur.fetchall(), columns=['movieId', 'cnt_rating'])

fig3 = px.histogram(movie_ratings,
                   x='cnt_rating',
                   nbins=30,
                   title='Numero de calificaciones por pelicula',
                   labels={'cnt_rating': 'Cantidad de calificaciones', 'count': 'Numero de peliculas'})

fig3.update_layout(
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False))

fig3.show()

El histograma muestra que la gran mayoria de las peliculas han recibido muy pocas calificaciones, concentrandose principalmente entre 0 y 20. Muy pocas peliculas han sido valoradas por muchos usuarios. Esto puede ser debido a una desigualdad en cuanto a la popularidad de estas

## 3.4 Distribución de películas por año

In [253]:
print("\nDistribución de Películas por Año:\n", df_movies_processed['year'].value_counts().sort_index())


Distribución de Películas por Año:
 year
1902.0      1
1903.0      1
1908.0      1
1915.0      1
1916.0      4
         ... 
2014.0    278
2015.0    271
2016.0    211
2017.0    143
2018.0     41
Name: count, Length: 106, dtype: int64


In [254]:
fig4 = px.histogram(df_movies_processed, x='year', title='Distribución de Películas por Año',
                                  labels={'year': 'Año', 'count': 'Número de Películas'})

fig4.update_layout(
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False))

fig4.show()

La gráfica muestra la evolución de la cantidad de películas producidas por año desde 1903 hasta 2018. En donde se observa que la mayoria de películas fueron publicadas en años recientes entre medidades de la decada del 90 y 2015, año a patir del cual se evidencia un decrecimiento acelerado.

## 3.5 Distribución de peliculas por década

In [255]:
df_movies_processed['decade'] = (df_movies_processed['year'] // 10 * 10).where(df_movies_processed['year'].notna())
decade_counts = df_movies_processed['decade'].value_counts().sort_index()
print("\nDistribución de Películas por Década:\n", decade_counts)


Distribución de Películas por Década:
 decade
1900.0       3
1910.0       7
1920.0      37
1930.0     136
1940.0     197
1950.0     278
1960.0     400
1970.0     499
1980.0    1176
1990.0    2207
2000.0    2849
2010.0    1915
Name: count, dtype: int64


In [256]:
fig5 = px.bar(decade_counts, x=decade_counts.index, y=decade_counts.values,
                            title='Distribución de Películas por Década',
                            labels={'index': 'Década', 'y': 'Número de Películas'})
fig5.update_layout(
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False))

fig5.show()

Entre las décadas de 1900 y 1920, la producción cinematográfica fue baja y relativamente estable, reflejando los inicios de la industria del cine. A partir de la década de 1970 se observa un crecimiento más marcado, que se intensifica notablemente entre las décadas de 1990 y 2010, alcanzando su punto máximo en la decada del 2000.

##3.6 Frecuencia de Géneros

In [257]:
genres_counts = df_movies_processed.drop(columns=['movieId', 'title', 'year', 'decade'], errors='ignore').sum().sort_values(ascending=False)
print("\nFrecuencia de Géneros:\n", genres_counts)


Frecuencia de Géneros:
 Drama          4361
Comedy         3756
Thriller       1894
Action         1828
Romance        1596
Adventure      1263
Crime          1199
Sci-Fi          980
Horror          978
Fantasy         779
Children        664
Animation       611
Mystery         573
Documentary     440
War             382
Musical         334
Western         167
IMAX            158
Film-Noir        87
dtype: int64


In [258]:
fig7 = px.bar(genres_counts, x=genres_counts.index, y=genres_counts.values,
                            title='Frecuencia de Géneros', labels={'y': 'Número de Películas', 'index': 'Género'})

fig7.update_layout(
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False))

fig7.show()

Los generos mas repetidos en las peliculas son los de Drama y Comedy, mientras que los que menos participación tiene son los de Western, IMAX y FIlm-Noir.

## 3.7 Número de calificaciones por película

In [259]:
n_top_movies = 10
print(f"\nTop {n_top_movies} Películas con Más Calificaciones (usando función):\n")
df_top_n_movies = top_n_peliculas_valoradas(cur)
print(df_top_n_movies)


Top 10 Películas con Más Calificaciones (usando función):

                                title  total_valoraciones  movieId
0                        Forrest Gump                 329      356
1           Shawshank Redemption, The                 317      318
2                        Pulp Fiction                 307      296
3           Silence of the Lambs, The                 279      593
4                         Matrix, The                 278     2571
5  Star Wars: Episode IV - A New Hope                 251      260
6                       Jurassic Park                 238      480
7                          Braveheart                 237      110
8          Terminator 2: Judgment Day                 224      589
9                    Schindler's List                 220      527


In [260]:
fig9 = px.bar(df_top_n_movies, x='title', y='total_valoraciones',
                          title=f'Top {n_top_movies} Películas con Más Calificaciones',
                          labels={'title': 'Título de la Película', 'total_valoraciones': 'Número de Valoraciones'})

fig9.update_layout(
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False))

fig9.show()

Forest Grump es la pelicula con mas calificaciones del catalogo con un total de 329, mientras que al final del top 10, con 109 calificaciones por detras, se encuenta Shindler´s List con 220 calificaciones.

# 4. GENERACIÓN DE TABLAS FINALES
---

Guardar los DataFrame como una tabla SQL

In [261]:
#Guardamos los procesamientos hechos en el df en la tabla
df_movies_processed.to_sql('movies', con, if_exists='replace', index=False)

9708

In [262]:
#La tabla final de ratings se va a crear con las películas con mas de 10 calificaciones y ratings entre 4 y 5
df_peliculas_filtradas = pd.read_sql("""
    SELECT m.movieId, m.title, m.year,
           m."Action", m."Adventure", m."Animation", m."Children", m."Comedy", m."Crime",
           m."Documentary", m."Drama", m."Fantasy", m."Film-Noir", m."Horror", m."IMAX",
           m."Musical", m."Mystery", m."Romance", m."Sci-Fi", m."Thriller", m."War", m."Western", m."decade",
           AVG(r.rating) AS avg_rating,
           COUNT(r.rating) AS num_ratings
    FROM movies m
    JOIN ratings r ON m.movieId = r.movieId
    WHERE (m."Action" + m."Adventure" + m."Animation" + m."Children" + m."Comedy" +
           m."Crime" + m."Documentary" + m."Drama" + m."Fantasy" + m."Film-Noir" +
           m."Horror" + m."IMAX" + m."Musical" + m."Mystery" + m."Romance" +
           m."Sci-Fi" + m."Thriller" + m."War" + m."Western") >= 1
      AND r.rating BETWEEN 4 AND 5
    GROUP BY m.movieId, m.title, m.year
    HAVING COUNT(r.rating) > 10
""", con)

df_peliculas_filtradas

,movieId,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,decade,avg_rating,num_ratings
0,1,Toy Story,1995.0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,1990.0,4.380952,147
1,2,Jumanji,1995.0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1990.0,4.210000,50
2,3,Grumpier Old Men,1995.0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1990.0,4.333333,18
3,5,Father of the Bride Part II,1995.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1990.0,4.291667,12
4,6,Heat,1995.0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1990.0,4.405797,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,152081,Zootopia,2016.0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,2010.0,4.326087,23
1100,164179,Arrival,2016.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,2010.0,4.411765,17
1101,166528,Rogue One: A Star Wars Story,2016.0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,2010.0,4.421053,19
1102,168252,Logan,2017.0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,2010.0,4.525000,20


In [263]:
df_peliculas_filtradas.to_sql('tabla_final', con, if_exists='replace', index=False)

1104

In [264]:
#Se crea df escalado
df_scaled = df_peliculas_filtradas.copy()

cols_a_escalar = [
    'year', 'decade',
    'avg_rating', 'num_ratings']

# Escalar variables
scaler = StandardScaler()
df_scaled[cols_a_escalar] = scaler.fit_transform(df_scaled[cols_a_escalar])

df_scaled.head()

,movieId,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,decade,avg_rating,num_ratings
0,1,Toy Story,0.105087,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0.069109,0.258499,3.754408
1,2,Jumanji,0.105087,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0.069109,-1.045428,0.586623
2,3,Grumpier Old Men,0.105087,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0.069109,-0.104712,-0.458419
3,5,Father of the Bride Part II,0.105087,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.069109,-0.422521,-0.654364
4,6,Heat,0.105087,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0.069109,0.448000,1.207117


In [265]:
# Crear carpeta de salida si no existe
os.makedirs("salidas", exist_ok=True)

# Guardar como joblib
joblib.dump(df_scaled, "salidas/df_scaled.joblib")

print("✅ Archivo guardado como salidas/df_scaled.joblib")

✅ Archivo guardado como salidas/df_scaled.joblib


In [266]:
# Especifica la ruta donde deseas guardar el archivo CSV
csv_file_path = '/content/drive/My Drive/cod/A3_marketing/data/ratings.csv'

# Convertir el DataFrame a CSV
df_ratings_processed.to_csv(csv_file_path, index=False)

print(f"El archivo CSV se ha guardado en: {csv_file_path}")

El archivo CSV se ha guardado en: /content/drive/My Drive/cod/A3_marketing/data/ratings.csv
